In [1]:
import torch
from torchvision import datasets, transforms
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import shutil
import os
from tqdm import tqdm
from PIL import Image, ImageDraw
import random

img_size = 64 # 64 lub 128

In [2]:
class AddRandomBlackRectangle:
    def __init__(self, min_size, max_size):
        self.min_size = min_size
        self.max_size = max_size

    def __call__(self, img):
        width, height = img.size
        rect_width = random.randint(self.min_size, self.max_size)
        rect_height = random.randint(self.min_size, self.max_size)
        x = random.randint(0, width - rect_width)
        y = random.randint(0, height - rect_height)
        
        # Create a new image with the same size as the original image
        rect_img = Image.new('RGB', img.size, (0, 0, 0))
        
        # Create a new mask image with the same size as the original image
        mask_img = Image.new('L', img.size, 0)
        
        # Draw a rectangle on the new image and the mask
        draw_img = ImageDraw.Draw(rect_img)
        draw_mask = ImageDraw.Draw(mask_img)
        draw_img.rectangle([x, y, x + rect_width, y + rect_height], fill="black")
        draw_mask.rectangle([x, y, x + rect_width, y + rect_height], fill=255)
        
        # Rotate the new image and the mask
        angle = random.randint(0, 90)
        rect_img = rect_img.rotate(angle)
        mask_img = mask_img.rotate(angle)
        
        # Paste the new image onto the original image using the mask
        img.paste(rect_img, (0, 0), mask_img)
        
        return img

In [5]:
# Define a transform to normalize the data
transform = transforms.Compose([
    AddRandomBlackRectangle(img_size//5, img_size//4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)), # to nam powinno dać wartości między -1 a 1
])

dataset_path_train = f'data/divided_x{img_size}/train'
dataset_path_val = f'data/divided_x{img_size}/val'
dataset_path_test = f'data/divided_x{img_size}/test'

In [6]:
train_dataset = datasets.ImageFolder(dataset_path_train, transform=transform)
val_dataset = datasets.ImageFolder(dataset_path_val, transform=transform)
test_dataset = datasets.ImageFolder(dataset_path_test, transform=transform)

print('train', len(train_dataset), 'val:', len(val_dataset), 'test:', len(test_dataset))

train 60579 val: 20194 test: 20196


In [7]:
train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [7]:
import os
from PIL import Image
import numpy as np
import pickle
from tqdm import tqdm

# Directory containing the images
image_dir = 'data/augmented_x64/val'

# Initialize an empty list to store the image arrays
image_arrays = []

# Traverse the directory tree
for root, dirs, files in os.walk(image_dir):
    for file in tqdm(files, desc='Loading images'):
        # Check if the file is an image (you can add more formats if needed)
        if file.endswith('.jpg'):
            # Construct the full file path
            file_path = os.path.join(root, file)
            # Open the image file, convert it to a numpy array, and append it to the list
            image_arrays.append(np.array(Image.open(file_path)))

# Serialize the list of numpy arrays with pickle
with open('data/augmented_x64_val.pkl', 'wb') as f:
    pickle.dump(image_arrays, f)

Loading images: 0it [00:00, ?it/s]
Loading images: 100%|██████████| 4998/4998 [00:39<00:00, 127.81it/s]


In [ ]:
import os
from PIL import Image
import numpy as np
import pickle
from tqdm import tqdm


def load_and_serialize_images(images_path: str) -> None:
    """
    Load images from a directory and serialize them into a pickle file.

    This function walks through the subdirectories 'val', 'train', and 'test' 
    in the given directory, loads all JPEG images, and serializes them into a 
    pickle file. The pickle file is saved in the parent directory of the given 
    directory, with the name in the format "{dir_name}_{subdir}.pkl".

    Parameters
    ----------
    images_path : str
        The path to the directory containing the images. This directory should 
        contain the subdirectories 'val', 'train', and 'test'.

    Returns
    -------
    None
    """

    subdirs = ['val', 'train', 'test']

    dir_name = os.path.basename(os.path.normpath(images_path))
    parent_dir = os.path.dirname(os.path.normpath(images_path))

    for subdir in subdirs:
        image_arrays = []
        subdir_path = os.path.join(images_path, subdir)

        for root, dirs, files in os.walk(subdir_path):
            for file in tqdm(files, desc=f'Loading images from {subdir}'):
                if file.endswith('.jpg'):
                    file_path = os.path.join(root, file)
                    image_arrays.append(np.array(Image.open(file_path)))

        # Serialize the list of numpy arrays with pickle
        pickle_file_path = f"{parent_dir}/{dir_name}_{subdir}.pkl"
        with open(pickle_file_path, 'wb') as f:
            pickle.dump(image_arrays, f)


load_and_serialize_images('data/augmented_x64')